# Initialize the Flickr API

In [1]:
from flickrapi import FlickrAPI

FLICKR_PUBLIC = '020b08efab13810efede65dedce49e46'
FLICKR_SECRET = '2d4a3834b368d9a9'

flickr = FlickrAPI(FLICKR_PUBLIC, FLICKR_SECRET, format='parsed-json')
extras='url_sq,url_t,url_s,url_q,url_m,url_n,url_z,url_c,url_l,url_o'

# Read the classes file

The classes file 'newclasses.txt' has a set of classes that we want to collect images for. The file 'availableclasses.txt' contains the names of the classes that we know When you want to add a new class. If you want to add new classes, add their names to newclasses.txt and run the script. God bless !


In [6]:
import os
import urllib.request

fname = 'newclasses.txt'
url_size = 'url_m'

def retrieve_list(fname_new, fname_available, numImg):
    newclasses = get_class_names(fname_new, fname_available)
    for cl in newclasses:
        getImages(cl, flickr, numImg)
        
def readFLineByLine(fname):
    with open(fname) as f:
        lList = f.readlines()
        # remove whitespace characters like `\n` at the end of each line
        lList = [x.strip() for x in lList]
        lList = list(filter(lambda x: x != '', lList))   # remove blanks
    f.close()
    return lList

def get_class_names(fname_new, fname_available):
    # index fname_available
    newclList = readFLineByLine(fname_new)
    oldclList = readFLineByLine(fname_available) 
    retrieveList = list(set(newclList) - set(oldclList))
    return retrieveList

def write_list_to_file(thefile, listname):
    for item in listname:
        thefile.write("%s\n" % item)

def getImages(classname, flickr, numImg):
    # create a folder first, the folders will be under the folder images -> image name
    cwd = os.getcwd() + '\\images'
    dirname = cwd  + '\\' + classname
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    page = 1
    urls = list()
    while len(urls)<numImg:
        urls.extend(flickr_search_urls(classname, url_size, page))
        page += 1
    thefile = open(dirname + '\\' + classname + '.txt', 'w')
    write_list_to_file(thefile, urls)
    # download te files
    download_images_from_url(dirname, urls)


def flickr_search_urls(keyword, url_size, pagen):
    """####################################################
        This searches for images in Flickr with keywords
                and returns images urls
    #######################################################
    """
    obj = flickr.photos.search(text=keyword, 
                            per_page=200, 
                            extras=url_size,
                              page = pagen)
    photos = obj['photos']['photo']
    url_list = map(lambda x: x.get(url_size), photos)
    clean_urls = filter(lambda x: x != None, url_list)
    return list(clean_urls)

def download_images_from_url(dirname, urls):
    for url, n in zip(urls, range(len(urls))):        
        urllib.request.urlretrieve(url, dirname + '\\' + str(n) + '.jpg')
        
retrieve_list('untitled.txt', 'old.txt', 1000)
    